In [16]:
!pip install --quiet scikit-surprise==1.1.1

import warnings
warnings.filterwarnings('ignore')

import math
import numpy as np
import pandas as pd

from surprise import KNNBasic
from surprise import accuracy
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split

In [2]:
knn_sims = np.array([0.8, 0.92, 0.75, 0.83])  # example array of similarity of user 2-5 to user 6
knn_ratings = np.array([3.0, 3.0, 2.0, 3.0])  # 2.0: audit; 3.0: completion

In [3]:
r_u6_ml = np.dot(knn_sims, knn_ratings) / sum(knn_sims)  # predicted rating of user 6 to item 'Machine Learning Capstone'
r_u6_ml

2.7727272727272725

In [4]:
true_rating = 3.0
rmse = math.sqrt(true_rating - r_u6_ml) ** 2
rmse

0.22727272727272751

In [5]:
# MOVIELENS-100K DATASET
data = Dataset.load_builtin('ml-100k', prompt=False)

train_set, test_set = train_test_split(data, test_size=.25)  # sample train and test set

In [6]:
algo = KNNBasic()
algo.fit(train_set)  # train algorithm
predictions = algo.test(test_set)
accuracy.rmse(predictions)

Computing the msd similarity matrix...


AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
# RATINGS DATASET
rating_url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML321EN-SkillsNetwork/labs/datasets/ratings.csv'
rating_df = pd.read_csv(rating_url)
rating_df.head()

In [ ]:
rating_sparse_df = rating_df.pivot(index='user', columns='item', values='rating').fillna(0).reset_index().rename_axis(index=None, columns=None)
rating_sparse_df.head()

In [ ]:
rating_df.to_csv('course_ratings.csv', index=False)

# read course rating dataset with columns 'user', 'item', 'rating'
reader = Reader(
        line_format='user item rating', sep=',', skip_lines=1, rating_scale=(2, 3))

In [ ]:
course_dataset = Dataset.load_from_file('course_ratings.csv', reader=reader)

train_set, test_set = train_test_split(course_dataset, test_size=.25)
print(f'{trainset.n_users} users and {trainset.n_items} items in train set')

In [ ]:
knn = KNNBasic(max_k=4, min_k=2, sim_option={
    'name':'cosine', 'user_based':False,
})
knn.fit(train_set)  # train KNN-based collaborative filtering model
predictions = knn.test(test_set)
accuracy.rmse(predictions)